In [1]:
import logging
import os

# set the directory to the location of the script
try:
    os.chdir("../../..")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


In [2]:
RUN_ID = "32005880-9dc1-408a-950c-e53ef24c648d"

## Access Cache (redis) 

In [3]:
from src.stateful.state_managment import (
    MemoManager,
)
from src.redis.manager import AzureRedisManager

REDIS_HOST=os.getenv("REDIS_HOST",)
REDIS_ACCESS_KEY=os.getenv("REDIS_ACCESS_KEY",)
REDIS_PORT=os.getenv("REDIS_PORT")

redis_manager = AzureRedisManager(access_key=REDIS_ACCESS_KEY, host=REDIS_HOST, port=REDIS_PORT)

[2025-08-20 14:05:23,363] INFO - src.redis.manager: Azure Redis connection initialized with access key.
INFO:src.redis.manager:Azure Redis connection initialized with access key.


In [4]:
cm = MemoManager.from_redis(RUN_ID, redis_manager)

In [54]:
# per-run summary
from src.tools.latency_helpers import PersistentLatency
pl = PersistentLatency(cm)

In [55]:
# raw data (JSON) for export/dashboards
cm.get_context("latency")

{'runs': {'d6661666bdc1': {'run_id': 'd6661666bdc1',
   'label': 'turn',
   'created_at': 514771.2507663,
   'samples': [{'stage': 'tts:synthesis',
     'start': 514771.3433764,
     'end': 514772.5951931,
     'dur': 1.2518166999798268,
     'meta': {'run_id': 'd6661666bdc1',
      'mode': 'acs',
      'voice': 'en-US-Ava:DragonHDLatestNeural',
      'style': 'chat',
      'rate': '+3%',
      'frames': 188}},
    {'stage': 'greeting_ttfb',
     'start': 514771.2508285,
     'end': 514772.6268811,
     'dur': 1.3760525999823585,
     'meta': {}},
    {'stage': 'tts:send_frames',
     'start': 514772.6268615,
     'end': 514772.6639884,
     'dur': 0.037126900046132505,
     'meta': {'run_id': 'd6661666bdc1', 'mode': 'acs', 'frames': 188}},
    {'stage': 'tts',
     'start': 514771.3433726,
     'end': 514774.4818586,
     'dur': 3.1384860000107437,
     'meta': {'run_id': 'd6661666bdc1',
      'mode': 'acs',
      'voice': 'en-US-Ava:DragonHDLatestNeural'}}]},
  '627bf5aa19ab': {'run_

In [56]:
# summary per run
rid = pl.current_run_id()  # or any run_id from CoreMemory["latency"]["order"]
print(pl.run_summary(rid))
print(cm.get_context("latency"))

{'tts:synthesis': {'count': 2, 'avg': 1.1212314000003971, 'min': 0.6663850999902934, 'max': 1.5760777000105008, 'total': 2.2424628000007942}, 'tts:send_frames': {'count': 2, 'avg': 0.004885299975285307, 'min': 0.0030033999937586486, 'max': 0.0067671999568119645, 'total': 0.009770599950570613}, 'tts': {'count': 2, 'avg': 1.2237607999995816, 'min': 0.7640026999870315, 'max': 1.6835189000121318, 'total': 2.4475215999991633}, 'general_agent': {'count': 1, 'avg': 4.742883300001267, 'min': 4.742883300001267, 'max': 4.742883300001267, 'total': 4.742883300001267}}
{'runs': {'d6661666bdc1': {'run_id': 'd6661666bdc1', 'label': 'turn', 'created_at': 514771.2507663, 'samples': [{'stage': 'tts:synthesis', 'start': 514771.3433764, 'end': 514772.5951931, 'dur': 1.2518166999798268, 'meta': {'run_id': 'd6661666bdc1', 'mode': 'acs', 'voice': 'en-US-Ava:DragonHDLatestNeural', 'style': 'chat', 'rate': '+3%', 'frames': 188}}, {'stage': 'greeting_ttfb', 'start': 514771.2508285, 'end': 514772.6268811, 'dur':

In [57]:
from src.tools.latency_analytics import compute_latency_statistics
stats = compute_latency_statistics(cm.get_context("latency"))

In [58]:
stats["charts"]["bar_stage_avg"]

[{'stage': 'claim_agent', 'avg_ms': 4728.035400010413, 'count': 2},
 {'stage': 'auth_agent', 'avg_ms': 4068.816500004484, 'count': 9},
 {'stage': 'general_agent', 'avg_ms': 4027.3189500148874, 'count': 2},
 {'stage': 'greeting_ttfb', 'avg_ms': 1376.0525999823585, 'count': 1},
 {'stage': 'tts', 'avg_ms': 1183.245619446906, 'count': 36},
 {'stage': 'tts:synthesis', 'avg_ms': 1039.0047888860054, 'count': 36},
 {'stage': 'tts:send_frames', 'avg_ms': 14.914388886407123, 'count': 36}]

In [49]:
stats["sla"]

{'runs': 2,
 'tts_max_single_le_1_5s_pct': 100.0,
 'greeting_ttfb_le_2_0s_pct': 50.0}

In [38]:
stats["threshold_breaches"]       

{'tts': [{'run_id': '381e0a295d3d',
   'duration': 1.934430799970869,
   'threshold': 1.5},
  {'run_id': 'c34b063d8b07', 'duration': 1.7204691999941133, 'threshold': 1.5},
  {'run_id': 'c34b063d8b07', 'duration': 2.9333323999890126, 'threshold': 1.5},
  {'run_id': 'af915be99264', 'duration': 2.0834055999876, 'threshold': 1.5},
  {'run_id': 'af915be99264', 'duration': 1.5783597999834456, 'threshold': 1.5},
  {'run_id': 'af915be99264', 'duration': 2.521651999966707, 'threshold': 1.5}]}

### Access Operational DB (Post Analyics) -> CosmosDB

In [ ]:
from src.cosmosdb.manager import CosmosDBMongoCoreManager

cosmos_manager = CosmosDBMongoCoreManager(
    connection_string=os.getenv("AZURE_COSMOS_CONNECTION_STRING"),
    database_name=os.getenv("AZURE_COSMOS_DB_DATABASE_NAME"),
    collection_name=os.getenv("AZURE_COSMOS_DB_COLLECTION_NAME"),
)

In [ ]:
query = {"session_id": "a23455d6"}
document = cosmos_manager.read_document(query)

if document:
    print("Found session:", document)
else:
    print("No session found.")